Uncomment all the code to run

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from geopy.point import Point
from geopy.geocoders import *
from geopy.distance import *
import gmaps

In [2]:
detroit_demolition_permits = pd.read_csv('detroit-demolition-permits.tsv',sep='\t')
detroit_blight_violations = pd.read_csv('detroit-blight-violations.csv')
detroit_311 = pd.read_csv('detroit-311.csv')
detroit_crime= pd.read_csv('detroit-crime.csv')

C:\Anaconda3\envs\gl-env\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Anaconda3\envs\gl-env\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def get_location(s):
    location = geolocator.geocode(s+' Detroit, MI, USA',timeout=300)
    print location.latitude, location.longitude
    return (location.latitude, location.longitude)
def process(s):
    if pd.isnull(s['site_location']) or s['site_location'][-1] != ')':
        return Point(get_location(s['SITE_ADDRESS']))
    else:
        return Point(s['site_location'].split('\n')[-1])
points = detroit_demolition_permits.apply(process,1)

42.39463 -83.202285
42.393245 -83.200561
42.4202690125 -82.9804840088
42.417018 -82.957717
42.402244 -83.266304
42.4352531433 -82.9825057983
42.420886 -82.975898
42.357276 -83.184295
42.324184 -83.117691
42.4173774719 -82.9567184448
42.3639411926 -83.0236358643
42.4429702759 -82.9649734497
42.4006576538 -82.9383773804
42.3370857239 -83.1274871826
42.3891601562 -82.9429779053
42.4444656372 -83.0644302368
42.443916 -83.049194
42.445735 -83.04927
42.445991 -83.049858
42.441337 -83.049049
42.400844574 -82.9507522583
42.4009437561 -82.9508132935
42.418136 -83.052894
42.41558 -83.253395
42.394618988 -83.1164169312
42.4383087158 -83.0311203003
42.4371414185 -83.0655288696
42.446685 -83.049308
42.445884 -83.049858
42.3405075073 -83.200592041
42.436256 -83.030426
42.4363632202 -83.064491272
42.436714 -83.065505
42.4369735718 -83.0649642944
42.4438018799 -83.0497817993
42.4013366699 -82.9510803223
42.445556 -83.049842
42.434829 -83.065383
42.447078 -83.049324
42.402755 -82.95272
42.4382095337 -8

In [8]:
detroit_demolition_permits['LAT'] = points.apply(lambda x: x.latitude)

In [9]:
detroit_demolition_permits['LON'] = points.apply(lambda x: x.longitude)

In [10]:
detroit_demolition_permits.to_json('detroit_demolition_permits_with_lat_lon.json')

In [11]:
def clean(s):
    if(pd.notnull(s['LAT'])):
        return True
    if pd.isnull(s['LOCATION']):
        return False
    if s['LOCATION'][-1] ==')':
        return True
    return False
detroit_crime = detroit_crime[detroit_crime.apply(clean,1)]

In [12]:
def get_points(s):
    if pd.notnull(s['LAT']):
        return Point((s['LAT'],s['LON']))
    else:
        return Point(s['LOCATION'].split('\n')[-1])
points = detroit_crime.apply(get_points,1)

In [13]:
detroit_crime['LAT'] = points.apply(lambda x: x.latitude)
detroit_crime['LON'] = points.apply(lambda x: x.longitude)

In [14]:
detroit_crime.to_json('detroit_crime_with_lat_lon.json')

In [15]:
detroit_crime.shape

(119901, 18)

In [16]:
points = detroit_blight_violations.ViolationAddress.apply(lambda x: Point(x.split('\n')[-1]))

In [17]:
detroit_blight_violations.head()

,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,...,LateFee,StateFee,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress,processed_address
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,...,$150.00,$10.00,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...","743 Beaubien\nDetroit, MI 48226\n(42.333730630...",2566 grand
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,...,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...","20501 HEYDEN\nDETROIT, MI 48219\n(42.442177633...",19014 ashton
2,19804,05000026DAH,Department of Public Works,"TALTON, CAROL ANN",18735,STAHELIN,18735,STAHELI N,DETROIT,MI,...,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...","18735 STAHELI N\nDETROIT, MI 48219\n(42.428707...",18735ahelin
3,20208,05000027DAH,Department of Public Works,"BONNER, DARRYL E.",20125,MONICA,25335,PEEKSKILL,SOUTHFIELD,MI,...,$10.00,$10.00,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...","25335 PEEKSKILL\nSOUTHFIELD, MI 48043\n(42.475...",20125 monica
4,20211,05000028DAH,Department of Public Works,"GREGORY, JAMES LEE",17397,PRAIRIE,17397,PRAIRIE,DETROIT,MI,...,$10.00,$10.00,$0.00,$140.00,PAID IN FULL,0.0,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...","17397 PRAIRIE\nDETROIT, MI 48221\n(42.42031769...",17397 prairie


In [18]:
detroit_blight_violations['LAT'] = points.apply(lambda x: x.latitude)
detroit_blight_violations['LON'] = points.apply(lambda x: x.longitude)

In [19]:
detroit_blight_violations.head()

,TicketID,TicketNumber,AgencyName,ViolName,ViolationStreetNumber,ViolationStreetName,MailingStreetNumber,MailingStreetName,MailingCity,MailingState,...,CleanUpCost,JudgmentAmt,PaymentStatus,Void,ViolationCategory,ViolationAddress,MailingAddress,processed_address,LAT,LON
0,26288,05000001DAH,Department of Public Works,"Group, LLC, Grand Holding",2566,GRAND BLVD,743,"Beaubien, Ste. 201",Detroit,MI,...,$0.00,$1680.00,PAID IN FULL,0.0,0,"2566 GRAND BLVD\nDetroit, MI\n(42.363182370000...","743 Beaubien\nDetroit, MI 48226\n(42.333730630...",2566 grand,42.363182,-83.091677
1,19800,05000025DAH,Department of Public Works,"JACKSON, RAECHELLE",19014,ASHTON,20501,HEYDEN,DETROIT,MI,...,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"19014 ASHTON\nDetroit, MI\n(42.429390762000025...","20501 HEYDEN\nDETROIT, MI 48219\n(42.442177633...",19014 ashton,42.429391,-83.220394
2,19804,05000026DAH,Department of Public Works,"TALTON, CAROL ANN",18735,STAHELIN,18735,STAHELI N,DETROIT,MI,...,$0.00,$140.00,PAID IN FULL,0.0,0,"18735 STAHELIN\nDetroit, MI\n(42.4287074590000...","18735 STAHELI N\nDETROIT, MI 48219\n(42.428707...",18735ahelin,42.428707,-83.227548
3,20208,05000027DAH,Department of Public Works,"BONNER, DARRYL E.",20125,MONICA,25335,PEEKSKILL,SOUTHFIELD,MI,...,$0.00,$140.00,NO PAYMENT APPLIED,0.0,0,"20125 MONICA\nDetroit, MI\n(42.44169828400004,...","25335 PEEKSKILL\nSOUTHFIELD, MI 48043\n(42.475...",20125 monica,42.441698,-83.145018
4,20211,05000028DAH,Department of Public Works,"GREGORY, JAMES LEE",17397,PRAIRIE,17397,PRAIRIE,DETROIT,MI,...,$0.00,$140.00,PAID IN FULL,0.0,0,"17397 PRAIRIE\nDetroit, MI\n(42.42031769500005...","17397 PRAIRIE\nDETROIT, MI 48221\n(42.42031769...",17397 prairie,42.420318,-83.145328


In [20]:
detroit_blight_violations.to_json('detroit_blight_violations_with_lat_lon.json')

In [21]:
detroit_311.head()

,ticket_id,city,issue_type,ticket_status,issue_description,rating,ticket_closed_date_time,acknowledged_at,ticket_created_date_time,ticket_last_updated_date_time,address,lat,lng,location,image,processed_address
0,1516722,City of Detroit,Clogged Drain,Acknowledged,"Two drains one on each side of street, street ...",3,NaN,03/06/2015 10:03:38 PM,03/06/2015 09:57:52 PM,04/12/2015 01:01:10 AM,"13120-13130 Ilene Street Detroit, MI 48238, USA",42.383998,-83.161039,"(42.3839977668, -83.1610385642)",NaN,13120-13130 ilene
1,1525361,City of Detroit,Clogged Drain,Acknowledged,standing water on lumplin,2,NaN,03/11/2015 04:23:11 PM,03/11/2015 04:14:29 PM,04/07/2015 02:04:44 PM,"1485 E. Outer Drive Detroit, Michigan",42.440471,-83.080919,"(42.4404708, -83.080919)",NaN,1485 e outer drive
2,1525218,City of Detroit,Clogged Drain,Closed,CITZEN CALLED TO REPORT CLOGGED DRAINS,2,08/15/2015 12:03:43 AM,03/11/2015 03:39:05 PM,03/11/2015 03:26:20 PM,08/15/2015 12:03:44 AM,"15460 Eastburn Detroit, Michigan",42.445244,-82.962038,"(42.445244, -82.962038)",NaN,15460 eastburn
3,1525214,City of Detroit,Clogged Drain,Acknowledged,Citizen called DWSD to report clogged drain,3,NaN,03/11/2015 03:35:02 PM,03/11/2015 03:22:42 PM,06/07/2015 10:07:48 PM,"17541 Mendota St Detroit, Michigan",42.421043,-83.166194,"(42.421043, -83.166194)",NaN,17541 mendota
4,1525142,City of Detroit,Clogged Drain,Acknowledged,@ THE CORNER OF GRIGGS & MARGARETA,2,NaN,03/11/2015 03:04:59 PM,03/11/2015 02:53:23 PM,03/11/2015 03:04:59 PM,"Griggs Detroit, Michigan",42.402033,-83.162874,"(42.4020334, -83.1628741)",NaN,griggs


In [22]:
points = detroit_311.apply(lambda x: Point(x['lat'],x['lng']),1)

In [23]:
detroit_311['LAT'] = points.apply(lambda x: x.latitude)
detroit_311['LON'] = points.apply(lambda x: x.longitude)

In [24]:
detroit_311.to_json('detroit_311_with_lat_lon.json')